# Visualizing Musical Taste
DSCI 624 - Data Visualization Final Project - Graham Bachman

## Install packages and establish Spotify Authorization Flow

In [ ]:
# Install Packages
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Setup Spotify Authorization Code Flow
username = '12128528664'
scope = 'user-library-read playlist-read-private'
cid = "352ccfcaee8542e8bd078171005ea1e1"
secret = "8bfdcd0803294d43a881c0e5df17be0b"
uri = "http://localhost:5006/"
scope = 'user-library-read playlist-read-private, user-top-read'

credentials = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=uri, scope=scope)
sp = spotipy.Spotify(auth_manager=credentials)

## Gathering Data: 
### Getting the ID's of all 'Your Year Wrapped' Playlists

In [ ]:
wrapped_playlists = {'Year':[2020,2019,2018,2017,2016], 
           'playlist_id':['37i9dQZF1EMcq0eGAv9Vuu','37i9dQZF1EteTmCePxInAP','37i9dQZF1Ejx9qPCOpDhGt','37i9dQZF1E9U9HDEkRUWR5','37i9dQZF1Cz4OxltUfbPLp']}

wrapped_playlists = pd.DataFrame.from_dict(wrapped_playlists)
len(wrapped_playlists)
wrapped_playlists

### Getting song and song info for each playlist

In [ ]:
wrapped_songs =  pd.DataFrame(columns=["list_id", "song_id","song_name","artist","popularity",'release_date']
                       )


for listid in wrapped_playlists["playlist_id"]:
    songs = []
    content = sp.user_playlist_tracks("12128528664", listid, fields=None, limit=100, offset=0, market=None)
    songs += content['items']
    for song in songs:
        wrapped_songs = wrapped_songs.append({"list_id" : listid,
                                  "song_id":song['track']['id'],
                                  "song_name":song['track']['name'],
                                  "artist":song['track']['artists'][0]['name'],
                                  "popularity": song['track']['popularity'],
                                  "release_date": song['track']['album']['release_date']},ignore_index=True)
len(wrapped_songs)
wrapped_songs

## Review of Spotify Audio Features
**Acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

**Danceability**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

**Energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

**Instrumentalness**: Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

**Liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides a strong likelihood that the track is live.

**Loudness**: the overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.

**Speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audiobook, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

**Valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

### Grab Audio Features for all songs

In [ ]:
wrapped_features = pd.DataFrame(columns=["song_id","energy", "liveness","tempo","speechiness",
                                "acousticness","instrumentalness","danceability",
                                "duration_ms","loudness","valence",
                                "mode","key"])
for song in wrapped_songs['song_id']:
    features = sp.audio_features(tracks = [song])[0]
    wrapped_features = wrapped_features.append({"song_id":song,
                                    "energy":features['energy'], 
                                    "liveness":features['liveness'],
                                    "tempo":features['tempo'],
                                    "speechiness":features['speechiness'],
                                    "acousticness":features['acousticness'],
                                    "instrumentalness":features['instrumentalness'],
                                    "danceability":features['danceability'],
                                    "duration_ms":features['duration_ms'],
                                    "loudness":features['loudness'],
                                    "valence":features['valence'],
                                    "mode":features['mode'],
                                    "key":features["key"],
                                 },ignore_index=True)

len(wrapped_features)
wrapped_features

### Merge into one Dataset

In [ ]:
wrapped_join = pd.merge(wrapped_features,wrapped_songs,how='left',left_on='song_id', right_on='song_id')
wrapped = pd.merge(wrapped_playlists, wrapped_join, how='left', left_on = 'playlist_id', right_on='list_id')
wrapped

### Normalize Loudness

In [ ]:
# We want to inspect following feature in radar chart: acousticness, danceability,energy, instrumentalness, loudness, valance.
# Besides loudness, all other features are in 0-1 scale, thus we first convert loudness to the scale of 0-1.

# create function to do linear transformation on variable to change value to [0,1]
def convert_scale(df,col):
    new_max = 1
    new_min = 0
    new_range = new_max-new_min
    max_val = df[col].max()
    min_val=df[col].min()
    val_range = max_val - min_val
    df[col+'_adj']=df[col].apply(lambda x: (((x-min_val)*new_range)/val_range)+new_min)
    return

convert_scale(wrapped,'loudness')
wrapped = wrapped[['Year', 'playlist_id', 'song_id', 'acousticness','danceability','energy','instrumentalness','valence','loudness_adj', 'speechiness', 
                  'liveness', 'key', 'mode', 'tempo', 'song_name', 'artist', 'popularity', 'release_date']]
wrapped

In [ ]:
wrapped.mean()

In [ ]:
avg = pd.DataFrame({'Year':['All'],
                    'playlist_id':['null'],
                    'song_id':['null'],
                    'acousticness':[.411829],
                    'danceability':[0.534604],
                    'energy':[0.529452],
                    'instrumentalness':[0.117031],
                    'valence':[0.370057],
                    'loudness_adj':[0.684628],
                    'speechiness':[0.097449],
                    'liveness':[0.186855],
                    'key':[4.778571],
                    'mode':[0.608929],
                    'tempo':[114.947016],
                    'song_name':['null'],
                    'artist':['null'],
                    'popularity':[45.926786],
                    'release_date': ['0000-00-00']
                   })
wrapped.append(avg)

### Create CSV File

In [ ]:
wrapped.to_csv(r'C:\Users\bachm\OneDrive - Maryville University\M.S\FA2020\DSCI 624 - Data Visualization\spotify_wrapped.csv', index = False)

## A Brief Look at Artist and Genres Through the Years

In [ ]:
results = sp.current_user_top_artists(time_range='long_term', limit=10)
for i, item in enumerate(results['items']):
    print(i, item['name'])
print()

print('hip-hop/rap = 5\npop = 3\nalt folk = 2')

In [ ]:
draw_radar(feature_mean.index.values[0],feature_mean.index.values[7],feature_mean.index.values[1],feature_mean.index.values[6],
           '37i9dQZF1EMcq0eGAv9Vuu')

## Getting Data for Full Streaming History

In [ ]:
import json
with open('StreamingHistory0.json', encoding='utf8') as f:
    data = json.load(f)

In [ ]:
spotify_data = pd.DataFrame()

def extract_json_value(column_name):
    
    return [i[column_name] for i in data]

spotify_data['artist_name'] = extract_json_value('artistName')
spotify_data['end_time'] = extract_json_value('endTime')
spotify_data['ms_played'] = extract_json_value('msPlayed')
spotify_data['track_name'] = extract_json_value('trackName')

In [ ]:
import spotipy.util as util

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=cid,   
                                   client_secret=secret,     
                                   redirect_uri=uri)

In [ ]:
import requests

# write the function to get track_id
def get_id(track_name: str,artist:str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    track_artist = track_name+ " " + artist
    params = [
    ('q',track_artist ),#q is the search query parameter
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 10)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

# Get track_id for streaming history    
spotify_data["track_id"] = spotify_data.apply(lambda x: get_id(x["track_name"],
                                                               x["artist_name"],
                                                                 token),axis=1)

In [ ]:
from collections import OrderedDict
spotify_data_nonull = spotify_data["track_id"].dropna()
spotify_data_nonull

track = list(OrderedDict.fromkeys(spotify_data_nonull))

In [ ]:
# get track's feature
my_feature = pd.DataFrame(columns=["song_id","energy", "liveness","tempo","speechiness",
                                "acousticness","instrumentalness","danceability",
                                "duration_ms","loudness","valence",
                                "mode","key"])
for song in track:
    features = sp.audio_features(tracks = [song])[0]
    if features is not None:
        my_feature = my_feature.append({"song_id":song,
                                    "energy":features['energy'], 
                                    "liveness":features['liveness'],
                                    "tempo":features['tempo'],
                                    "speechiness":features['speechiness'],
                                    "acousticness":features['acousticness'],
                                    "instrumentalness":features['instrumentalness'],
                                    "danceability":features['danceability'],
                                    "duration_ms":features['duration_ms'],
                                    "loudness":features['loudness'],
                                    "valence":features['valence'],
                                    "mode":features['mode'],
                                    "key":features["key"],
                                 },ignore_index=True)
    else:
        pass

In [ ]:
my_streaming= pd.merge(spotify_data,my_feature,how="left",left_on= "track_id", right_on="song_id")
my_streaming.drop(my_streaming[my_streaming["ms_played"]==0].index,inplace=True)
my_streaming

In [ ]:
my_streaming.to_csv(r'C:\Users\bachm\OneDrive - Maryville University\M.S\FA2020\DSCI 624 - Data Visualization\spotify_streaming.csv', index = False)